In [7]:
import calendar
import pandas as pd
import panel as pn

from datetime import date, timedelta
from sqlalchemy import create_engine

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
today

datetime.date(2022, 1, 16)

In [8]:
cols = 'trade name qty price active reason market'.split()
colt = 'trade name qty target active reason market'.split()

In [9]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite)

,id,trade,name,qty,price,active,reason,market
0,693,B,SMT,18000,5.5,0,,SET


### After call Ord-Log.bat which calculates spreads between target price and SET price.

In [10]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name
#col_names   = ['trade','name','spreads','reason','market','qty','target_price','set_price','chg_amt','chg_pct','active']
df_log = pd.read_csv(input_file, sep=',',  index_col=None)
df_log

,trade,name,spd,reason,market,qty,target,current,change,percent,active
0,B,AEONTS,-7,1L,SET100,1200,184.00,187.50,-0.50,-0.27%,0
1,B,BGC,-5,DOS,SET,10000,10.00,10.50,-0.10,-0.94%,0
2,B,KCE,-4,RD05pct,SET50,800,81.00,82.00,-2.25,-2.67%,0
3,B,NOBLE,-10,RD15%,SET,21000,5.85,6.35,-0.05,-0.78%,0
4,B,PTT,-9,RD05%,SET50,3000,36.50,38.75,-0.50,-1.27%,0
5,B,PTTGC,-5,RD10pct,SET50,1600,59.25,60.50,-0.50,-0.82%,0
6,B,RATCH,-7,RD05%,SET50,3000,42.75,44.50,-0.25,-0.56%,0
7,B,RCL,-2,RU15pct,SET100,1500,50.00,50.50,-2.75,-5.16%,0
8,B,ROJNA,-7,6L,SET,18000,6.30,6.65,0.00,0.00%,0
9,B,SCC,-5,RD05pct,SET50,300,384.00,389.00,-1.00,-0.26%,0


In [11]:
def calc(vals):
    spreads = vals
    if abs(spreads) < 10:
        return 1
    else:
        return 0

In [12]:
df_log['active'] = df_log['spd'].apply(calc)

In [13]:
df_log.groupby('active')['trade'].count()

active
0    10
1    20
Name: trade, dtype: int64

In [14]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

31

In [15]:
df_log.dtypes

trade       object
name        object
spd          int64
reason      object
market      object
qty          int64
target     float64
current    float64
change     float64
percent     object
active       int64
dtype: object

In [16]:
final = df_log[colt]
final

,trade,name,qty,target,active,reason,market
0,B,AEONTS,1200,184.00,1,1L,SET100
1,B,BGC,10000,10.00,1,DOS,SET
2,B,KCE,800,81.00,1,RD05pct,SET50
3,B,NOBLE,21000,5.85,0,RD15%,SET
4,B,PTT,3000,36.50,1,RD05%,SET50
5,B,PTTGC,1600,59.25,1,RD10pct,SET50
6,B,RATCH,3000,42.75,1,RD05%,SET50
7,B,RCL,1500,50.00,1,RU15pct,SET100
8,B,ROJNA,18000,6.30,1,6L,SET
9,B,SCC,300,384.00,1,RD05pct,SET50


In [17]:
rcds = final.values.tolist()
rcds

[['B', 'AEONTS', 1200, 184.0, 1, '1L', 'SET100'],
 ['B', 'BGC', 10000, 10.0, 1, 'DOS', 'SET'],
 ['B', 'KCE', 800, 81.0, 1, 'RD05pct', 'SET50'],
 ['B', 'NOBLE', 21000, 5.85, 0, 'RD15%', 'SET'],
 ['B', 'PTT', 3000, 36.5, 1, 'RD05%', 'SET50'],
 ['B', 'PTTGC', 1600, 59.25, 1, 'RD10pct', 'SET50'],
 ['B', 'RATCH', 3000, 42.75, 1, 'RD05%', 'SET50'],
 ['B', 'RCL', 1500, 50.0, 1, 'RU15pct', 'SET100'],
 ['B', 'ROJNA', 18000, 6.3, 1, '6L', 'SET'],
 ['B', 'SCC', 300, 384.0, 1, 'RD05pct', 'SET50'],
 ['B', 'SMT', 18000, 5.5, 0, nan, 'SET'],
 ['B', 'STA', 2500, 28.0, 0, 'RD15%', 'SET100'],
 ['B', 'STARK', 24000, 4.24, 0, '6L', 'SET100'],
 ['B', 'TQM', 4500, 46.0, 1, '3L', 'SET100'],
 ['B', 'TSE', 40000, 2.6, 1, '6L', 'SET'],
 ['B', 'TVO', 3600, 30.0, 1, 'DOS', 'SET100'],
 ['B', 'TYCN', 8000, 3.44, 0, 'RD05pct', 'SET'],
 ['S', 'CKP', 10000, 5.5, 0, '10pct', 'SET100'],
 ['S', 'EPG', 3000, 12.9, 1, '15pct', 'SET100'],
 ['S', 'GLOBAL', 2500, 21.0, 1, '5pct', 'SET50'],
 ['S', 'IVL', 3000, 50.5, 1, '20pct'

In [18]:
for rcd in rcds:
    print(rcd)

['B', 'AEONTS', 1200, 184.0, 1, '1L', 'SET100']
['B', 'BGC', 10000, 10.0, 1, 'DOS', 'SET']
['B', 'KCE', 800, 81.0, 1, 'RD05pct', 'SET50']
['B', 'NOBLE', 21000, 5.85, 0, 'RD15%', 'SET']
['B', 'PTT', 3000, 36.5, 1, 'RD05%', 'SET50']
['B', 'PTTGC', 1600, 59.25, 1, 'RD10pct', 'SET50']
['B', 'RATCH', 3000, 42.75, 1, 'RD05%', 'SET50']
['B', 'RCL', 1500, 50.0, 1, 'RU15pct', 'SET100']
['B', 'ROJNA', 18000, 6.3, 1, '6L', 'SET']
['B', 'SCC', 300, 384.0, 1, 'RD05pct', 'SET50']
['B', 'SMT', 18000, 5.5, 0, nan, 'SET']
['B', 'STA', 2500, 28.0, 0, 'RD15%', 'SET100']
['B', 'STARK', 24000, 4.24, 0, '6L', 'SET100']
['B', 'TQM', 4500, 46.0, 1, '3L', 'SET100']
['B', 'TSE', 40000, 2.6, 1, '6L', 'SET']
['B', 'TVO', 3600, 30.0, 1, 'DOS', 'SET100']
['B', 'TYCN', 8000, 3.44, 0, 'RD05pct', 'SET']
['S', 'CKP', 10000, 5.5, 0, '10pct', 'SET100']
['S', 'EPG', 3000, 12.9, 1, '15pct', 'SET100']
['S', 'GLOBAL', 2500, 21.0, 1, '5pct', 'SET50']
['S', 'IVL', 3000, 50.5, 1, '20pct', 'SET50']
['S', 'KBANK', 800, 150.5, 0, 

In [19]:
sql = """
INSERT INTO orders (trade, name, qty, price, active, reason, market)
VALUES (?, ?, ?, ?, ?, ?, ?)
"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active, reason, market)
VALUES (?, ?, ?, ?, ?, ?, ?)



In [20]:
for rcd in rcds:
    conlite.execute(sql, rcd)

In [21]:
sql = '''
SELECT *
FROM orders
ORDER BY trade, name'''
df_orders = pd.read_sql(sql, conlite)
df_tab = pn.widgets.Tabulator(df_orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=    id trade  ..., width=800)

In [22]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)